In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hithadhoo,-0.6000,73.0833,82.63,74,97,12.80,MV,2022-07-19 00:38:44
1,1,Acapulco,16.8634,-99.8901,85.82,74,40,11.50,MX,2022-07-19 00:38:44
2,2,Rikitea,-23.1203,-134.9692,74.68,74,0,10.67,PF,2022-07-19 00:38:44
3,3,Nouadhibou,20.9310,-17.0347,73.38,78,77,9.22,MR,2022-07-19 00:38:45
4,4,Lokosovo,61.1333,74.8167,61.00,69,95,7.16,RU,2022-07-19 00:38:45


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [10]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
clouds = city_data_df["Cloudiness"]
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap of percent humidity
wind = city_data_df["Wind Speed"]
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)
clouds = city_data_df["Cloudiness"]
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hithadhoo,-0.6000,73.0833,82.63,74,97,12.80,MV,2022-07-19 00:38:44
1,1,Acapulco,16.8634,-99.8901,85.82,74,40,11.50,MX,2022-07-19 00:38:44
5,5,Kahului,20.8947,-156.4700,87.82,61,40,26.46,US,2022-07-19 00:38:45
7,7,Bengkulu,-3.8004,102.2655,78.57,79,13,3.09,ID,2022-07-19 00:38:46
11,11,Hamilton,39.1834,-84.5333,79.57,82,40,4.61,US,2022-07-19 00:36:27
13,13,Siocon,7.7061,122.1353,79.57,83,100,3.78,PH,2022-07-19 00:38:48
14,14,Tautira,-17.7333,-149.1500,76.23,72,19,4.83,PF,2022-07-19 00:38:48
20,20,Georgetown,5.4112,100.3354,78.75,91,40,5.01,MY,2022-07-19 00:35:27
24,24,Vaini,-21.2000,-175.2000,78.96,83,40,11.50,TO,2022-07-19 00:38:50
25,25,Dzaoudzi,-12.7887,45.2699,76.89,65,75,20.71,YT,2022-07-19 00:38:51


In [16]:
preferred_cities_df.count()

City_ID       206
City          206
Lat           206
Lng           206
Max Temp      206
Humidity      206
Cloudiness    206
Wind Speed    206
Country       206
Date          206
dtype: int64

In [17]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Hithadhoo,MV,82.63,-0.6000,73.0833,
1,Acapulco,MX,85.82,16.8634,-99.8901,
5,Kahului,US,87.82,20.8947,-156.4700,
7,Bengkulu,ID,78.57,-3.8004,102.2655,
11,Hamilton,US,79.57,39.1834,-84.5333,
13,Siocon,PH,79.57,7.7061,122.1353,
14,Tautira,PF,76.23,-17.7333,-149.1500,
20,Georgetown,MY,78.75,5.4112,100.3354,
24,Vaini,TO,78.96,-21.2000,-175.2000,
25,Dzaoudzi,YT,76.89,-12.7887,45.2699,


In [18]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [19]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'AeJbb3dE52scPWDAEnK6rBJwAy6MNIYJ9UA6floQNAiw-QyarlSHgvb5mQ9jcsU3rMXSOHch5KjapCus7TF3O-KmBLVPshnnTCrHIAc8zyeFEydV4lKQ7N31SaG-dVTfLeK86s8FRzVS6U7nqDa1eAvLH8d7niJ8QH2gbHCXdwbwRA3pP_5tnw5SFgP0-GDfOEAKrXBuhE4NdJ_47SMyGvUdbL57v38QC0_klKs9Iuh3Fl4fYaQtD_bvJxP2VOnt5ux5Qw2cLBRZXbgPU0d24sYycozSwMGCKAjRqVOjvp5OA2yloUk-8R_JM4hq3ol0Cmu0JUg7rXut4lOm7gS4UOThDUn1QJJXbpxACSRYfLlWV5ZpuQ3vEkPNwALJ0q8WyPIucxUqVR5--qaOZjvqG5EkaSchCkRMgHAc1ahvTTwQfcRSvkbnj77sr-Pb',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [20]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

IndexError: list index out of range

In [22]:
# Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

Hotel not found... skipping.


In [23]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [26]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))